<a href="https://colab.research.google.com/github/amun83/Koscom-Financial-Data-Bootcamp/blob/master/FD104_Oilprice_com_News_Text_Mining_and_One_More_Thing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting News Articles from https://oilprice.com/

In [0]:
ON_COLAB  = True
DATA_PATH = ''

In [0]:
!pip install -q --upgrade pandas_datareader
!pip install -q --upgrade finance-datareader
!pip install -q --upgrade html5lib
!pip install -q --upgrade lxml
!pip install -q --upgrade seaborn
!pip install -q --upgrade gspread

In [0]:
# utilities
import datetime
import re
import csv
from IPython.display import display

# web scraping
import requests
import urllib.parse
from bs4 import BeautifulSoup as bs
import pandas_datareader as pdr
import FinanceDataReader as fdr

# Math/Stat
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
#import math

# Machine Learning
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

# visualization
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline

In [49]:
if ON_COLAB :
  from google.colab import drive
  drive.mount('/gdrive')
  DATA_PATH = '/gdrive/My Drive/oilprice.com/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
pattern = re.compile(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d\d),\s(\d{4})')
# Example: By Julianne Geiger - Dec 14, 2018, 8:00 PM CST

In [0]:
def get_news_urls_by_page(page_num):
  page_url = 'https://oilprice.com/Latest-Energy-News/World-News/Page-' + str(page_num) + '.html'
  # https://oilprice.com/Latest-Energy-News/World-News/Page-865.html

  try:
    r = requests.get(page_url)
    print('Request(GET) >> OK: ' + page_url)
  except:
    print('Request(GET) >> Failed: ' + page_url)
    return None

  soup = bs(r.text)
  items = soup.find_all('div', {"class":"categoryArticle__content"})

  if not items:
    print('Request(GET) >> Empty Page: ' + page_url)
    return None
  
  return [item.find('a', href = True)['href'] for item in items]

In [52]:
news_urls =[]

for i in range(1, 1000):
  urls = get_news_urls_by_page(i)
  if not urls:
    break
    
  news_urls += urls

Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-1.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-2.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-3.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-4.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-5.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-6.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-7.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-8.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-9.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-10.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-11.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-12.html
R

In [0]:
with open(DATA_PATH + 'oilprice.com_news_urls.csv', 'w') as f:
  fw = csv.writer(f)
  
  for url in news_urls:
    fw.writerow([url])

In [0]:
def get_news_content(url):
  try:
    get_news_content.counter += 1
  except:
    get_news_content.counter = 0
  
  try:
    r = requests.get(url)
    print('[' + str(get_news_content.counter) + '] Request(GET) >> OK: ' + url)
  except:
    print('[' + str(get_news_content.counter) + '] Request(GET) >> Failed: ' + url)
    return None

  soup = bs(r.text)

  title = soup.find('h1').text

  date = soup.find('span', {'class':'article_byline'}).text

  match = re.search(pattern, date)
  mmm, dd, yyyy = match.groups(0)
  date = yyyy + "-" + mmm + "-" + dd

  raw_content = soup.find('div', {'id':'news-content'}).find_all('p')
  content = ' '.join([p.text.strip() for p in raw_content[:-2]])

  return date, title, content

In [0]:
news = [get_news_content(url) + (url,) for url in news_urls]

Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/France-Extends-State-Guarantee-For-Total-Participated-Russian-LNG-Projects.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Saudi-Oil-Minister-Discusses-Further-Cooperation-With-Worlds-Largest-Refiner.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Pemex-Boosts-2019-Budget-14-With-Focus-On-Producing-Fields.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Environment-Conscious-Quebecois-Buy-More-Gas-Guzzlers.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Climate-Change-Conference-Strikes-Agreement-On-Rulebook.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Cuadrilla-Halts-Fracking-Again-Operations-As-Tremors-Persist.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Tesla-Cuts-Prices-In-China-As-Beijing-Suspends-New-US-Car-Tariffs.html
Request(GET) >> OK: 

In [0]:
file_prefix = DATA_PATH + 'oilprice.com_news.csv'

with open(DATA_PATH + 'oilprice.com_news.csv', 'w') as f:
  fw = csv.writer(f) 
  
  for url in news_urls:
    try:
      r = requests.get(url)
      print('Requested(GET) >> OK: ' + url)
    except:
      print('Request(GET) >> Failed: ' + url)
      break

    soup = bs(r.text)

    title = soup.find('h1').text

    date = soup.find('span', {'class':'article_byline'}).text

    match = re.search(pattern, date)
    mmm, dd, yyyy = match.groups(0)
    date = yyyy + "-" + mmm + "-" + dd

    raw_content = soup.find('div', {'id':'news-content'}).find_all('p')
    content = ' '.join([p.text.strip() for p in raw_content[:-2]])

    #print('- Title:   ' + title)
    #print('- Date:    ' + date)
    #print('- Content: ' + content)
    
    fw.writerow([date, title, content, url])